# Parse New Gene Table

**from:** Maria D. Vibranovski

Here attached is a list from Yong Zhang group based on our paper from 2010. But this is a still not published updated version that he shared with me but you can use.

If you need details about the columns, please look at https://genome.cshlp.org/content/suppl/2010/08/27/gr.107334.110.DC1/SupplementalMaterial.pdf  table 2a.

But mainly, what you need to select is the child genes with:

gene_type = D or R or DL or RL
m_type= M
note that contains "chrX-"

D and R stands for DNA-based Duplication and RNA-based duplication
L means that the assignment of the parental genes is less reliable.
M indicates that is between chromosome movement.

Hope it helps. If you need I can parse for you. please, do not hesitate to ask. But I thought you would prefer a complete list where you can look at subsets.

cheers

Maria


In [1]:
import os
import sys
from pathlib import Path
import re

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact, chi2_contingency
from scipy.stats.contingency import margins
import statsmodels.formula.api as smf

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../lib')
from larval_gonad.notebook import Nb
from larval_gonad.plotting import make_figs
from larval_gonad.config import memory

In [2]:
# Setup notebook
nbconfig = Nb.setup_notebook(seurat_dir='../output/scrnaseq-wf/scrnaseq_combine_force')

last updated: 2019-04-12 
Git hash: 8b2294f0f1fb95fa1db81581d215310271068158


In [3]:
def adjusted_residuals(observed, expected):
    resid = (observed - expected) / np.sqrt(expected)
    n = observed.sum().sum()
    rsum, csum = margins(observed)
    v = csum * rsum * (n - rsum) * (n - csum) / n**3
    return (observed - expected) / np.sqrt(v)

## Import data from Maria

## FBgn sanitizer

I don't know where these FBgns are from, so I need to sanitize them to my current annotation.

In [4]:
assembly = nbconfig.assembly
tag = nbconfig.tag
pth = Path(os.environ['REFERENCES_DIR'], f'{assembly}/{tag}/fb_annotation/{assembly}_{tag}.fb_annotation')

# Create an FBgn 
mapper = {}

for record in pd.read_csv(pth, sep='\t').to_records():
    mapper[record.primary_FBgn] = record.primary_FBgn
    
    try:
        for g in record.secondary_FBgn.split(','):
            mapper[g] = record.primary_FBgn
    except AttributeError:
        pass

In [5]:
autosomes = ['chr2L', 'chr2R', 'chr3L', 'chr3R']

In [6]:
movement = (
    pd.read_excel('../data/external/maria/dm6_ver78_genetype.new.xlsx')
    .query('gene_type == ["D", "R", "Dl", "Rl"] and m_type == "M"')
    .assign(child_chrom = lambda df: df.note.str.extract('(chr.*?)-'))
    .assign(parent_chrom = lambda df: df.note.str.extract('-(chr.*?)[:;]'))
    .assign(FBgn = lambda df: df.child_id.map(mapper))
    .assign(parent_FBgn = lambda df: df.parent_id.map(mapper))
    .drop(['child_id', 'parent_id', 'note', 'm_type'], axis=1)
    .dropna()
    .set_index('FBgn')
    .assign(moved_x_to_a = lambda df: (df.parent_chrom == 'chrX') & df.child_chrom.isin(autosomes))
    .assign(moved_a_to_a = lambda df: df.parent_chrom.isin(autosomes) & df.child_chrom.isin(autosomes))
    .assign(moved_a_to_x = lambda df: df.parent_chrom.isin(autosomes) & (df.child_chrom == 'chrX'))
    .query('moved_x_to_a | moved_a_to_a | moved_a_to_x')
)

movement.head()

,gene_type,child_chrom,parent_chrom,parent_FBgn,moved_x_to_a,moved_a_to_a,moved_a_to_x
FBgn,,,,,,,
FBgn0000246,Dl,chr3R,chrX,FBgn0002873,True,False,False
FBgn0002562,D,chrX,chr2L,FBgn0002563,False,False,True
FBgn0003060,D,chr3R,chr2L,FBgn0032282,False,True,False
FBgn0003086,Dl,chrX,chr3L,FBgn0036393,False,False,True
FBgn0003357,R,chr3R,chr3L,FBgn0250815,False,True,False


In [7]:
biomarkers = (
    nbconfig.seurat.get_biomarkers('res.0.6')
    .cluster.map(nbconfig.short_cluster_annot)
    .pipe(lambda x: x[x != 'UNK'])
    .to_frame()
    .reset_index()
    .groupby('FBgn')
    .apply(lambda x: '|'.join(x.cluster))
    .rename('biomakrer_cluster')
)

In [8]:
germ_comp = (
    pd.read_csv('../output/scrnaseq-wf/germcell_deg/gonia_vs_mid.tsv', sep='\t')
    .assign(FBgn = lambda df: df.primary_FBgn)
    .assign(SP = lambda df: df.avg_logFC > 0)
    .assign(M1 = lambda df: df.avg_logFC < 0)
    .set_index('FBgn')
    .loc[:, ['SP', 'M1']]
    .idxmax(axis=1)
    .rename('bias_gonia_vs_mid')
)

In [9]:
biomarkers.head()

FBgn
FBgn0000014       PC|TE
FBgn0000015          TE
FBgn0000017    TE|PC|LC
FBgn0000024          PC
FBgn0000028         E1º
Name: biomakrer_cluster, dtype: object

In [10]:
df = (
    movement.join(biomarkers, how='left')
    .join(germ_comp.rename('bias_gonia_vs_mid_child'), how='left')
    .join(germ_comp.rename('bias_gonia_vs_mid_parent'), on='parent_FBgn', how='left')
)

In [11]:
tpm = (
    pd.read_parquet('../output/scrnaseq-wf/tpm.parquet')
    .assign(cluster = lambda df: df.cluster.map(nbconfig.short_cluster_annot))
    .pivot_table(index='FBgn', columns='cluster', values='TPM')
    .loc[:, ['SP', 'M1º']]
)

In [12]:
df = (
    df
    .join(tpm['SP'].rename('SP_child'), how='left')
    .join(tpm['M1º'].rename('M1_child'), how='left')
    .join(tpm['SP'].rename('SP_parent'), on='parent_FBgn', how='left')
    .join(tpm['M1º'].rename('M1_parent'), on='parent_FBgn', how='left')
)

In [13]:
df.head(100)

,gene_type,child_chrom,parent_chrom,parent_FBgn,moved_x_to_a,moved_a_to_a,moved_a_to_x,biomakrer_cluster,bias_gonia_vs_mid_child,bias_gonia_vs_mid_parent,SP_child,M1_child,SP_parent,M1_parent
FBgn,,,,,,,,,,,,,,
FBgn0000246,Dl,chr3R,chrX,FBgn0002873,True,False,False,NaN,NaN,NaN,74.507506,40.811514,0.652152,1.751364
FBgn0002562,D,chrX,chr2L,FBgn0002563,False,False,True,NaN,NaN,NaN,0.023132,0.061855,0.144521,0.137125
FBgn0003060,D,chr3R,chr2L,FBgn0032282,False,True,False,NaN,NaN,NaN,0.021381,0.000000,0.075548,0.126258
FBgn0003086,Dl,chrX,chr3L,FBgn0036393,False,False,True,NaN,NaN,NaN,NaN,NaN,0.000000,0.057476
FBgn0003357,R,chr3R,chr3L,FBgn0250815,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FBgn0004414,Rl,chr3L,chr2L,FBgn0283658,False,True,False,NaN,NaN,NaN,0.043373,0.695865,4.892144,10.511107
FBgn0010549,Dl,chr2R,chr3R,FBgn0039645,False,True,False,NaN,NaN,NaN,0.035092,0.223757,0.000000,0.061723
FBgn0011559,Dl,chr2L,chr3L,FBgn0052133,False,True,False,NaN,NaN,NaN,8.682675,22.154939,6.796721,1.752841
FBgn0015008,Rl,chr3R,chrX,FBgn0025633,True,False,False,E1º|L1º|M1º,M1,NaN,11.194324,212.349137,31.173193,4.033355


In [14]:
df

,gene_type,child_chrom,parent_chrom,parent_FBgn,moved_x_to_a,moved_a_to_a,moved_a_to_x,biomakrer_cluster,bias_gonia_vs_mid_child,bias_gonia_vs_mid_parent,SP_child,M1_child,SP_parent,M1_parent
FBgn,,,,,,,,,,,,,,
FBgn0000246,Dl,chr3R,chrX,FBgn0002873,True,False,False,NaN,NaN,NaN,74.507506,40.811514,0.652152,1.751364
FBgn0002562,D,chrX,chr2L,FBgn0002563,False,False,True,NaN,NaN,NaN,0.023132,0.061855,0.144521,0.137125
FBgn0003060,D,chr3R,chr2L,FBgn0032282,False,True,False,NaN,NaN,NaN,0.021381,0.000000,0.075548,0.126258
FBgn0003086,Dl,chrX,chr3L,FBgn0036393,False,False,True,NaN,NaN,NaN,NaN,NaN,0.000000,0.057476
FBgn0003357,R,chr3R,chr3L,FBgn0250815,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FBgn0004414,Rl,chr3L,chr2L,FBgn0283658,False,True,False,NaN,NaN,NaN,0.043373,0.695865,4.892144,10.511107
FBgn0010549,Dl,chr2R,chr3R,FBgn0039645,False,True,False,NaN,NaN,NaN,0.035092,0.223757,0.000000,0.061723
FBgn0011559,Dl,chr2L,chr3L,FBgn0052133,False,True,False,NaN,NaN,NaN,8.682675,22.154939,6.796721,1.752841
FBgn0015008,Rl,chr3R,chrX,FBgn0025633,True,False,False,E1º|L1º|M1º,M1,NaN,11.194324,212.349137,31.173193,4.033355


In [15]:
out_order = [
    'child_chrom',
    'parent_chrom',
    'parent_FBgn',
    'gene_type',
    'moved_x_to_a',
    'moved_a_to_a',
    'moved_a_to_x',
    'biomakrer_cluster',
    'bias_gonia_vs_mid_child',
    'bias_gonia_vs_mid_parent',
    'SP_child',
    'M1_child',
    'SP_parent',
    'M1_parent'
]

df.reindex(columns=out_order).reset_index().rename({'FBgn': 'child_FBgn'}, axis=1).fillna('nan').to_csv('../output/notebook/2019-03-30_movement_data.csv', index=None)

In [16]:
print('\n'.join(out_order))

child_chrom
parent_chrom
parent_FBgn
gene_type
moved_x_to_a
moved_a_to_a
moved_a_to_x
biomakrer_cluster
bias_gonia_vs_mid_child
bias_gonia_vs_mid_parent
SP_child
M1_child
SP_parent
M1_parent
